<a href="https://colab.research.google.com/github/O-Kpy/Dacon/blob/main/Dacon_%E1%84%8C%E1%85%A1%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%8B%E1%85%A5_%E1%84%80%E1%85%B5%E1%84%87%E1%85%A1%E1%86%AB_%E1%84%80%E1%85%B5%E1%84%92%E1%85%AE%E1%84%80%E1%85%B5%E1%84%89%E1%85%AE%E1%86%AF%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_AI_%E1%84%80%E1%85%A7%E1%86%BC%E1%84%8C%E1%85%B5%E1%86%AB%E1%84%83%E1%85%A2%E1%84%92%E1%85%AC_%E1%84%8C%E1%85%A1%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%8B%E1%85%A5_%E1%84%80%E1%85%B5%E1%84%87%E1%85%A1%E1%86%AB_%E1%84%80%E1%85%B5%E1%84%92%E1%85%AE%E1%84%80%E1%85%B5%E1%84%89%E1%85%AE%E1%86%AF%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_AI_%E1%84%80%E1%85%A7%E1%86%BC%E1%84%8C%E1%85%B5%E1%86%AB%E1%84%83%E1%85%A2%E1%84%92%E1%85%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip install transformers
!pip install sentencepiece

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  37896      0 --:--:-- --:--:-- --:--:-- 37896
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 2.7 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.277 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatibl

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 312 kB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 448 kB 67.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from konlpy.tag import Okt
import re

import torch_xla
import torch_xla.core.xla_model as xm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import KFold, StratifiedKFold

import transformers
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import XLMRobertaConfig, XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

import warnings
warnings.filterwarnings('ignore')

In [6]:
train = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/자연어 기반 기후기술분류 AI 경진대회/자연어 기반 기후기술분류 AI 경진대회_train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/자연어 기반 기후기술분류 AI 경진대회/자연어 기반 기후기술분류 AI 경진대회_test.csv')
submit = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/자연어 기반 기후기술분류 AI 경진대회/자연어 기반 기후기술분류 AI 경진대회_sample_submission.csv')

# 데이터 전처리

In [7]:
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]", " ", sent) #특수문자 및 기타 제거
    sent_clean = re.sub(' +', ' ', sent_clean) # 다중 공백 제거
    return sent_clean

def data_preprocessing(data):
    data = data.fillna('NONE')
    data['요약문_연구목표'] = data.apply(lambda x : x['과제명'] if x['요약문_연구목표'] == 'NONE' else x['요약문_연구목표'], axis=1)
    
    okt = Okt()
    data['요약문_한글키워드'] = data.apply(lambda x : ','.join(okt.nouns(x['과제명'])) if x['요약문_한글키워드'] == 'NONE' else x['요약문_한글키워드'], axis = 1)
    
    return data

In [8]:
train = data_preprocessing(train)
test = data_preprocessing(test)

for i in train.select_dtypes('object').columns:
  if i == 'label':
    continue
  
  train[i] = train[i].apply(lambda x: clean_text(x))

for i in test.select_dtypes('object').columns:
  test[i] = test[i].apply(lambda x: clean_text(x))

In [9]:
sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

y_train = train['label']

train_list = []
val_list = []

for i, (train_index, valid_index) in enumerate(sk.split(train, y_train)):
  df_train , df_valid = train.iloc[train_index], train.iloc[valid_index]

  train_list.append(df_train)  
  val_list.append(df_valid)

# Data 준비 (BERT version)

In [10]:
MODEL_TYPE = 'bert-base-multilingual-uncased'

NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 3 

L_RATE = 1e-5
MAX_LEN = 256
NUM_EPOCHS = 1
BATCH_SIZE = 32

device = xm.xla_device()
print(device)

xla:1


In [11]:
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE,
                                          do_lower_case=True)

class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, '사업명']
    sentence2 = self.df_data.loc[index, '사업_부처명']
    sentence3 = self.df_data.loc[index, '계속과제여부']
    sentence4 = self.df_data.loc[index, '내역사업명']
    sentence5 = self.df_data.loc[index, '과제명']
    sentence6 = self.df_data.loc[index, '요약문_연구목표']
    sentence7 = self.df_data.loc[index, '요약문_연구내용']
    sentence8 = self.df_data.loc[index, '요약문_기대효과']
    sentence9 = self.df_data.loc[index, '요약문_한글키워드']
    sentence10 = self.df_data.loc[index, '요약문_영문키워드']
    sentence = (sentence1, sentence2, sentence3, sentence4, sentence5, sentence6, sentence7, sentence8, sentence9, sentence10)

    encoded_dict = tokenizer.encode_plus(sentence,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         truncation=True,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]
    target = torch.tensor(self.df_data.loc[index, 'label'])

    sample = (input_ids, attention_mask, token_type_ids, target)

    return sample

  def __len__(self):
    return len(self.df_data)

class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, '사업명']
    sentence2 = self.df_data.loc[index, '사업_부처명']
    sentence3 = self.df_data.loc[index, '계속과제여부']
    sentence4 = self.df_data.loc[index, '내역사업명']
    sentence5 = self.df_data.loc[index, '과제명']
    sentence6 = self.df_data.loc[index, '요약문_연구목표']
    sentence7 = self.df_data.loc[index, '요약문_연구내용']
    sentence8 = self.df_data.loc[index, '요약문_기대효과']
    sentence9 = self.df_data.loc[index, '요약문_한글키워드']
    sentence10 = self.df_data.loc[index, '요약문_영문키워드']
    sentence = (sentence1, sentence2, sentence3, sentence4, sentence5, sentence6, sentence7, sentence8, sentence9, sentence10)

    encoded_dict = tokenizer.encode_plus(sentence,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         truncation=True,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    sample = (input_ids, attention_mask, token_type_ids)

    return sample

  def __len__(self):
    return len(self.df_data)

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [12]:
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
test = test.reset_index(drop=True)

In [13]:
train = CompDataset(df_train)
valid = CompDataset(df_valid)
test = TestDataset(test)

train_dataloader = DataLoader(train,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=4)

val_dataloader = DataLoader(valid,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=4)
test_dataloader = DataLoader(test,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=4)

In [14]:
model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                      num_labels=df_train['label'].nunique(),
                                      output_attentions=False,
                                      output_hidden_states=False)
model.to(device)

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [16]:
import random
import gc

seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

fold_val_acc_list = []
fold_val_f1_list = []
for i in range(0, NUM_FOLDS):
  fold_val_acc_list.append([])
  fold_val_f1_list.append([])

for epoch in range(0, NUM_FOLDS):
  print('\nNum folds used for training', NUM_FOLDS_TO_TRAIN)
  print(f'========= Epoch {epoch+1} / {NUM_EPOCHS} =========')

  num_folds = len(train_list)

  epoch_acc_score_list = []
  epoch_f1_score_list = []

  for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
    print('\n== Fold Model', fold_index)

    if epoch == 0:
      model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                            num_labels=df_train['label'].nunique(),
                                                            output_attentions=False,
                                                            output_hidden_states=False)
      model.to(device)
      optimizer = AdamW(model.parameters(), lr=L_RATE)

    else:
      path_model = 'model_' + str(fold_index) + '.bin'
      model.load_state_dict(torch.load(path_model))
      model.to(device)
    
    df_train = train_list[fold_index]
    df_val = val_list[fold_index]

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)

    train_data = CompDataset(df_train)
    val_data = CompDataset(df_val)

    train_dataloader = torch.utils.data.DataLoader(train_data,
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True,
                                                   num_workers=4)
    val_dataloader = torch.utils.data.DataLoader(val_data,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True,
                                                 num_workers=4)
    
    stacked_val_labels = []
    targets_list = []

    print('Training...')
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0

    for i, (batch) in enumerate(train_dataloader):
      input_ids = batch[0].to(device)
      attention_mask = batch[1].to(device)
      token_type_ids = batch[2].to(device)
      target = batch[3].to(device)

      model.zero_grad()
      
      output = model(input_ids, 
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids,
                      labels=target)
      
      loss = output[0]
      total_train_loss = total_train_loss + loss.item()

      optimizer.zero_grad()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      xm.optimizer_step(optimizer=optimizer, barrier=True)

    print(f'train loss : {total_train_loss/NUM_FOLDS_TO_TRAIN}')

    print('\nValidation')
    model.eval()

    torch.set_grad_enabled(False)

    total_val_loss=0

    for j, val_batch in enumerate(val_dataloader):
      input_ids = val_batch[0].to(device)
      attention_mask = val_batch[1].to(device)
      token_type_ids = val_batch[2].to(device)
      target = val_batch[3].to(device)

      output = model(input_ids,
                     attention_mask=attention_mask,
                     token_type_ids=token_type_ids,
                     labels=target)
      
      loss = output[0]
      total_val_loss = total_val_loss + loss.item()

      preds = output[1]

      val_preds = preds.detach().cpu().numpy()
      targets_np = target.to('cpu').numpy()

      targets_list.extend(targets_np)

      if j == 0:
        stacked_val_preds = val_preds

      else:
        stacked_val_preds = np.vstack((stacked_val_preds,
                                       val_preds))
    
    # calculate val acc for this fold
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)

    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred, average='macro')

    epoch_acc_score_list.append(val_acc)
    epoch_f1_score_list.append(val_f1)

    print('Val loss:', total_val_loss)
    print('Val Acc:', val_acc)
    print('Val f1:', val_f1)

    if epoch == 0:
      model_name = 'model_' + str(fold_index) + '.bin'
      torch.save(model.state_dict(), model_name)
      print('Saved model as', model_name)

    if epoch != 0:
      val_f1_list = fold_val_f1_list[fold_index]
      best_val_f1 = max(val_f1_list)

      if val_f1 > best_val_f1:
        model_name = 'model_' + str(fold_index) + '.bin'
        torch.save(model.state_dict(), model_name)
        print('Val f1 improved, saved model...')

    fold_val_acc_list[fold_index].append(val_acc)
    fold_val_f1_list[fold_index].append(val_f1)
    
    gc.collect()

  cv_f1 = sum(epoch_f1_score_list) / NUM_FOLDS_TO_TRAIN
  print('\nCV macro-F1:', cv_f1)


Num folds used for training 3
========= Epoch 1 / 1 =========

== Fold Model 0


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
train loss : 1576.5073986550171

Validation
Val loss: 1167.6063205003738
Val Acc: 0.8179627664151917
Val f1: 0.01956233603723258
Saved model as model_0.bin

== Fold Model 1


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...


KeyboardInterrupt: ignored

# Data 준비 (XLM-RoBERTa version)